In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from random import randint
from typing import Annotated, Any

from pydantic import Field
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

from agent_framework import Agent, tool
from agent_framework.orchestrations import SequentialBuilder
from agent_framework import Message, WorkflowEvent, Executor, WorkflowContext, handler, AgentExecutorResponse

In [ ]:
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

In [ ]:
@tool(approval_mode="never_require")
async def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    await asyncio.sleep(randint(0, 10) / 10.0)  # Simulate a network call
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

In [ ]:

credential = AzureCliCredential()
chat_client = AzureOpenAIResponsesClient(
    project_endpoint=project_endpoint,
    deployment_name=model,
    credential=credential,
)

weather_agent = chat_client.as_agent(
    instructions=(
        "You are a weather assistant. Provide the current weather conditions for a given location."
    ),
    name="weather_agent",
    id="weather_agent",
    tools=[get_weather],
)

writer = chat_client.as_agent(
    instructions=(
        "You are a concise copywriter. Provide a single, punchy marketing sentence based on the prompt."
    ),
    name="writer",
    id="writer",
)

reviewer = chat_client.as_agent(
    instructions=(
        "You are a thoughtful reviewer. Give brief feedback on the previous assistant message."
    ),
    name="reviewer",
    id="reviewer",
)

In [ ]:
class Summarizer(Executor):
    """Simple summarizer: consumes full conversation and appends an assistant summary."""

    @handler
    async def summarize(
        self,
        response: AgentExecutorResponse,
        ctx: WorkflowContext[list[Message]]
    ) -> None:
        conversation = response.full_conversation
        users = sum(1 for m in conversation if m.role == "user")
        assistants = sum(1 for m in conversation if m.role == "assistant")
        summary = Message(
            role="assistant",
            contents=[f"Summary -> users:{users} assistants:{assistants}"]
        )
        
        await ctx.send_message(list(conversation) + [summary])

In [ ]:
summarizer = Summarizer(id="summarizer")

In [ ]:
workflow = SequentialBuilder(
    participants=[writer, reviewer, weather_agent, summarizer]).build()

In [ ]:
output_evt: WorkflowEvent | None = None

query = "Write a tagline for a budget-friendly eBike, along with current weather in New York City."
async for event in workflow.run(query, stream=True):
    if event.type == "output":
        output_evt = event

In [ ]:
if output_evt:
    print("===== Final Conversation =====")
    messages: list[Message] | Any = output_evt.data
    for i, msg in enumerate(messages, start=1):
        name = msg.author_name or ("assistant" if msg.role == "assistant" else "user")
        print(f"{'-' * 60}\n{i:02d} [{name}]\n{msg.text}")